In [1]:
from qkeras import *
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Softmax, Add, ZeroPadding2D, MaxPooling2D
from tensorflow.keras.backend import random_uniform as rand_tensor
import numpy as np
from qkeras import *
import math
from tensorflow import keras

c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
Widths = [3, 10, 32, 60, 100]
Heights = [3, 10, 32, 60, 100]
pSize = [(1,1), (2,2), (3,3)]
pStride = [(1,1), (1,2), (2,1), (2,2), (2,3), (3,2), (3,3)]
mode = ["same", "valid"]


def myPooling2D(pool_type, y_arr, Pool_size, Stride, padding_type):
    assert len(Pool_size)==2 and len(Stride)==2, f"{len(Pool_size)}, {len(Stride)}"
    assert padding_type in {"same", "valid"}
    assert pool_type in {"max", "avg"}

    YN, YH, YW, YC = y_arr.shape
    PKH, PKW = Pool_size
    PSH, PSW = Stride

    if padding_type=="same":
        PXH = (YH+PSH-1)//PSH
        PXW = (YW+PSW-1)//PSW
    else:
        PXH = (YH-PKH+PSH)//PSH
        PXW = (YW-PKW+PSW)//PSW


    x_arr = np.zeros((YN, PXH, PXW, YC))

    p_st, q_st = 0, 0
    if padding_type == "same":
        p_st = max((PSH*(PXH-1)+PKH-YH)//2, 0)
        q_st = max((PSW*(PXW-1)+PKW-YW)//2, 0)


    for n in range(YN):
        for c in range(YC):
            for ixh in range(PXH):
                for ixw in range(PXW):
                    
                    ph_end, pw_end = PSH*ixh-p_st+PKH-1, PSW*ixw-q_st+PKW-1

                    result = -math.inf if pool_type == 'max' else 0
                    count = 0
                    
                    for iyh in range(ph_end, ph_end-PKH,-1):
                        for iyw in range(pw_end, pw_end-PKW,-1):
                            if iyh >=0 and iyw>=0 and iyh < YH and iyw < YW:
                                count += 1
                                
                                if pool_type=='max':
                                    result = max(result, y_arr[n][iyh][iyw][c])
                                else:
                                    result += y_arr[n,iyh,iyw,c]

                    x_arr[n,ixh,ixw,c] = result if pool_type=='max' else result/count
    return x_arr

n = 1
c = 1
for w in Widths:
    for h in Heights:
        for size in pSize:
            for stride in pStride:
                for m in mode:
                    assert size[0]==size[1], f"pooling size must be square!"
                    x_in = tf.random.uniform(shape=(n, h, w, c), minval=-100, maxval=100, dtype=tf.int32)
                    input_shape = x_in.shape[1:-1] + (c,)
                
                    x = Input(input_shape, name='input')
                    x1 = MaxPooling2D(size, strides=stride, padding=m)(x)
                    model1 = Model(inputs=[x], outputs=[x1])                        
                    y_ref = model1(x_in).numpy()
                    y_np = myPooling2D("max", x_in.numpy(), size, stride, m)
                
                    assert y_ref.all() == y_np.all(), f"maxpool error! shape = ({m}, {size}, {stride}, {n}, {h}, {w}, {c}), {y_ref.shape}, {y_np.shape}, {x_in.numpy()}, {y_ref}, {y_np}"
                    
                    x2 = AveragePooling2D(size, strides=stride, padding=m)(x)
                    model2 = Model(inputs=[x], outputs=[x2])
                    y_ref = model2(x_in).numpy()
                    y_np = myPooling2D("avg", x_in.numpy(), size, stride, m)
                    assert y_ref.all() == y_np.all(), f"avgpool error! shape = ({m}, {size}, {stride}, {n}, {h}, {w}, {c}), {y_ref.shape}, {y_np.shape}, {x_in.numpy()}, {y_ref}, {y_np}"
                    